In [33]:
import pandas as pd
import json
from itertools import product, chain
import random

In [34]:
# Opening JSON file
f = open('inputs/edars_calibrated.json', encoding="utf8")

# returns JSON object as
# a dictionary
edars_calibrated = json.load(f)

# Closing file
f.close()

In [35]:
# Opening JSON file
f = open('inputs/nuclis_no_sanejats.json', encoding="utf8")

# returns JSON object as
# a dictionary
nuclis_no_sanejats = json.load(f)

# Closing file
f.close()

In [59]:
edars_calibrated

{'ES9080010001010E': {'eu_code': 'ES9080010001010E',
  'dc_code': 'DABR',
  'nom': 'ABRERA',
  'population_real': 63029,
  'configuration': ['P', 'SP', 'O3', 'SF', 'UV'],
  'industriesTotalInfluent': {'q': 5069.401123287671,
   'Nitrogen Total': 127660.8132751089,
   'Nitrats': 5323.179287458064,
   'Amoni': 112367.3073360543,
   'Nitrogen orgànic': 11690.64413127975,
   'DBO 5 dies': 717857.9375566536,
   'Nitrogen Kjeldahl': 319834.2161643836,
   'Ciprofloxacina': 0.013104927506849315,
   'Sulfametoxazol': 0.005546755461187214,
   'Trimetoprim': 0.03621178914063928,
   'Venlafaxina': 0.1947901287237443,
   'Etilbenzè': 0.25688372240867585,
   'Fluorurs': 427.54900821917806,
   'Seleni': 1.9152613351598178,
   'Terbutilazina': 6.159786301369863e-05,
   'Toluè': 35.5344350254589,
   'Tricloroetilè': 0.002927532657534246,
   'Alaclor': 0.0,
   'Aldrin': 0.0,
   'Antracè': 0.01288689608219178,
   'Atrazina': 7.751827397260274e-05,
   'Benzo(b)fluorantè': 2.3825013698630138e-05,
   'Clorp

In [37]:
nuclis_no_sanejats

{'pt0008': {'poblacio_no_sanejada': 500,
  'lat': 42.25428085111,
  'lon': 3.07216058211613,
  'contaminants': {},
  'ubicacio_descarrega_actual': 'ES9431370002010E',
  'possibles_ubicacions_descarregues': ['pt0008', 'pt0010']},
 'pt0010': {'poblacio_no_sanejada': 1000,
  'lat': 42.2438402552367,
  'lon': 2.97448587690088,
  'contaminants': {'Ciprofloxacina': 0.2295579686233767,
   'Clotrimazol': 3.141558156,
   'Fluconazol': 0.0,
   'Imazalil': 0.01944115438333333,
   'Miconazol': 0.0,
   'Sulfametoxazol': 1.499127403846154e-05,
   'Trimetoprim': 1.474189652027972,
   'Venlafaxina': 0.9605064750551471,
   '2-etilhelix': 0.0,
   'Clorobenzè': 0.0,
   'Diclorobenzè': 0.0,
   'Etilbenzè': 0.0,
   'Fluorurs': 0.0781324,
   'Metolaclor': 0.0,
   'Seleni': 0.0,
   'Terbutilazina': 0.0,
   'Toluè': 0.0,
   'Tricloroetilè': 0.0,
   'Alaclor': 0.0,
   'Aldrin': 0.0,
   'Antracè': 0.0,
   'Atrazina': 0.0,
   'Benzo(b)fluorantè': 0.0,
   'Cloroalcans': 0.0,
   'Clorpirifòs': 0.0,
   'Diclorvós':

In [38]:
column_names = list(chain(*[[key+"_tech", key+'_pop'] for key in edars_calibrated.keys()])) + list(chain(*[[key+"_tech", key+'_pop'] for key in nuclis_no_sanejats.keys()]))
df = pd.DataFrame(columns=column_names)

In [39]:
tractaments_primaris = ['P']
tractaments_secundaris = ['SP', 'SN', 'SC']

#Si es modifica s'hauran de canviar els índexs de la variable terciaris_combinacio
tractaments_terciaris = [None, ['SF'], ["O3", "SF"], ['SF', 'UV'], ["O3","SF", "UV"], ["GAC"], ["O3","GAC","UV"], ["UF","RO","AOP"], ["UF", "UV"]]


In [40]:
terciaris_combinacio = {
    frozenset(['UF']): [tractaments_terciaris[7]],
    frozenset(['UF', 'CL']): [tractaments_terciaris[7], tractaments_terciaris[8]],
    frozenset(['CL']): [tractament for tractament in tractaments_terciaris[1:9]],
    frozenset(['UV', 'CL']): [tractaments_terciaris[3], tractaments_terciaris[6], tractaments_terciaris[8]],
    frozenset(['UF', 'SV']): [],
    frozenset(['UF', 'OTHER']): [],
    frozenset(['UF', 'UV', 'CL', 'OTHER']): [],
    frozenset(['UF', 'CL, OTHER']): [],
    frozenset(['SF']): [tractaments_terciaris[2], tractaments_terciaris[3], tractaments_terciaris[4]],
    frozenset(['SF', 'O3']): [tractaments_terciaris[4]],
    frozenset(['SF', 'UV']): [tractaments_terciaris[4]],
    frozenset(['GAC']): [tractaments_terciaris[6]],
}

In [41]:
#Cartesian product with lists as index, product
tractaments = list(enumerate(product(tractaments_primaris, tractaments_secundaris, tractaments_terciaris)))

In [42]:
#append no_mod as last element
no_mod = (len(tractaments), 'no_mod')

In [43]:
tractaments

[(0, ('P', 'SP', None)),
 (1, ('P', 'SP', ['SF'])),
 (2, ('P', 'SP', ['O3', 'SF'])),
 (3, ('P', 'SP', ['SF', 'UV'])),
 (4, ('P', 'SP', ['O3', 'SF', 'UV'])),
 (5, ('P', 'SP', ['GAC'])),
 (6, ('P', 'SP', ['O3', 'GAC', 'UV'])),
 (7, ('P', 'SP', ['UF', 'RO', 'AOP'])),
 (8, ('P', 'SP', ['UF', 'UV'])),
 (9, ('P', 'SN', None)),
 (10, ('P', 'SN', ['SF'])),
 (11, ('P', 'SN', ['O3', 'SF'])),
 (12, ('P', 'SN', ['SF', 'UV'])),
 (13, ('P', 'SN', ['O3', 'SF', 'UV'])),
 (14, ('P', 'SN', ['GAC'])),
 (15, ('P', 'SN', ['O3', 'GAC', 'UV'])),
 (16, ('P', 'SN', ['UF', 'RO', 'AOP'])),
 (17, ('P', 'SN', ['UF', 'UV'])),
 (18, ('P', 'SC', None)),
 (19, ('P', 'SC', ['SF'])),
 (20, ('P', 'SC', ['O3', 'SF'])),
 (21, ('P', 'SC', ['SF', 'UV'])),
 (22, ('P', 'SC', ['O3', 'SF', 'UV'])),
 (23, ('P', 'SC', ['GAC'])),
 (24, ('P', 'SC', ['O3', 'GAC', 'UV'])),
 (25, ('P', 'SC', ['UF', 'RO', 'AOP'])),
 (26, ('P', 'SC', ['UF', 'UV']))]

In [82]:
def edar_scenarios(row, edars_calibrated, tractaments, no_mod, terciaris_combinacio):

    #For each wwwtp in edars_calibrated, select a possible treatment
    for key, edar in edars_calibrated.items():
        #edar = edars_calibrated['ES9082790004050E']
        configuracio = edar['configuration']
        poblacio = edar['population_real']

        primari = configuracio[0] if len(configuracio) > 0 else None
        secundari = configuracio[1] if len(configuracio) > 1 else None
        terciaris = configuracio[2:] if len(configuracio) > 2 else None

        configuracions_possibles = tractaments

        if secundari is not None:
            configuracions_possibles = filter(lambda x: x[1][1] == secundari, configuracions_possibles)

        if terciaris is not None:
            configuracions_possibles = filter(lambda x: frozenset(terciaris) in terciaris_combinacio and x[1][2] in terciaris_combinacio[frozenset(terciaris)], configuracions_possibles)

            #Per cada terciari, afegir tecnologia no existent al tren però si a configuració actual (e.g. (1, ('P', 'SP', ['SF'])) és el mateix tren de tractament que (1, ('P', 'SP', ['SF', 'CL']))
            configuracions_possibles = [(configuracio[0], (configuracio[1][0], configuracio[1][1], configuracio[1][2] + [x for x in terciaris if x not in configuracio[1][2]])) for configuracio in configuracions_possibles]


        #Remove current configuration from possible configurations (as we will add no_mod category)
        try:
            configuracions_possibles = list(filter(lambda x: (x[1][0] != primari) or (x[1][1] != secundari) or (frozenset(x[1][2]) != frozenset(terciaris)), configuracions_possibles))
        except:
            #In case terciaris or x[1][2] is None
            configuracions_possibles = list(filter(lambda x: (x[1][0] != primari) or (x[1][1] != secundari) or (x[1][2] != terciaris), configuracions_possibles))


        configuracions_possibles = list(configuracions_possibles) + [no_mod]
        tractament_random = random.choice(list(map(lambda x: x[0], configuracions_possibles)))
        row[key+"_tech"] = tractament_random
        row[key+"_pop"] = poblacio


    return row

#df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


In [83]:
def nuclis_no_sanejats_scenarios(row, nuclis_no_sanejats, tractaments, no_mod):
    for item, nucli in nuclis_no_sanejats.items():

        poblacio = nucli['poblacio_no_sanejada']
        ubicacions_descarrega = nucli['possibles_ubicacions_descarregues']

        ubicacions_descarrega_random = random.choice(ubicacions_descarrega)
        configuracions_possibles = tractaments + [no_mod]
        tractament_random = random.choice(list(map(lambda x: x[0], configuracions_possibles)))

        row[item+"_tech"] = tractament_random
        row[ubicacions_descarrega_random+"_pop"] += poblacio


    return row

In [85]:
n_samples = 100
samples = []

for i in range(n_samples):
    new_row = dict.fromkeys(column_names, 0)
    new_row = edar_scenarios(new_row, edars_calibrated, tractaments, no_mod, terciaris_combinacio)
    new_row = nuclis_no_sanejats_scenarios(new_row, nuclis_no_sanejats, tractaments, no_mod)
    samples.append(new_row)

df = pd.DataFrame(samples)
df

,ES9080010001010E_tech,ES9080010001010E_pop,ES9080140001010E_tech,ES9080140001010E_pop,ES9430050001010E_tech,ES9430050001010E_pop,ES9430090001010E_tech,ES9430090001010E_pop,ES9170080001010E_tech,ES9170080001010E_pop,...,ES9083060001010E_tech,ES9083060001010E_pop,ES9081810001010E_tech,ES9081810001010E_pop,ES9172330003010E_tech,ES9172330003010E_pop,pt0008_tech,pt0008_pop,pt0010_tech,pt0010_pop
0,27,63029,17,4626,19,4611,20,1790,5,5569,...,3,3820,5,10032,26,2956,22,0,19,1500
1,27,63029,13,4626,20,4611,20,1790,6,5569,...,4,3820,4,10032,20,2956,26,1000,8,500
2,27,63029,11,4626,26,4611,23,1790,3,5569,...,6,3820,3,10032,27,2956,5,0,0,1500
3,27,63029,27,4626,21,4611,20,1790,8,5569,...,27,3820,6,10032,19,2956,22,0,21,1500
4,27,63029,13,4626,20,4611,20,1790,7,5569,...,8,3820,27,10032,27,2956,12,1500,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,27,63029,17,4626,23,4611,19,1790,7,5569,...,2,3820,5,10032,22,2956,19,1500,24,0
96,27,63029,14,4626,24,4611,27,1790,6,5569,...,7,3820,2,10032,24,2956,20,500,25,1000
97,27,63029,10,4626,23,4611,23,1790,4,5569,...,6,3820,6,10032,25,2956,20,1500,17,0
98,27,63029,11,4626,26,4611,19,1790,3,5569,...,27,3820,6,10032,21,2956,12,0,26,1500
